## Обучение с помощью одного нейрона

В этом блокноте мы построим нейрон, который классифицирует изображение как яблоко или банан, используя несколько функций из изображения. Мы будем **тренировать** наш нейрон, используя данные из многих изображений, которые уже были правильно классифицированы; тем самым наш нейрон **изучит**, какие параметры использовать, минимизируя функцию потерь с использованием градиентного спуска. Мы сделаем это с почти самым простым нейроном, а именно с тем, который принимает только два входа:

In [ ]:
using LinearAlgebra
include("draw_neural_net.jl")
# plotly()
number_inputs, number_neurons = 2, 1
draw_network([number_inputs, number_neurons])

Для этого нам нужно поработать и *очистить* некоторые реальные данные. Давайте начнем!

## Загрузка данных

Давайте загрузим кое-какие реальные данные! Мы будем использовать данные, которые мы подготовили из фотографий яблок и бананов; оказывается, что они хранится на диске в файлах данных как «значения, разделенные табуляцией». Мы можем прочитать эти данные с помощью пакета `CSV.jl` следующим образом.

In [ ]:
import Pkg; 
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("TextParse")

In [ ]:
using CSV
using TextParse

In [ ]:
applecols, applecolnames = TextParse.csvread("data/Apple_Golden_1.dat", '\t')
bananacols, bananacolnames = TextParse.csvread("data/bananas.dat", '\t');

In [ ]:
applecols

In [ ]:
typeof(applecols)

In [ ]:
applecolnames

Далее мы хотим использовать `DataFrames` для хранения информации из наших CSV-файлов.

In [ ]:
using DataFrames

In [ ]:
apples = DataFrame([col for col in applecols], [Meta.parse(ac) for ac in applecolnames])

In [ ]:
bananas = DataFrame([col for col in bananacols], [Meta.parse(bc) for bc in bananacolnames])

или же загрузить готовые Датафрэймы:

In [ ]:
# apples = CSV.read("data/Apple_Golden_1.csv")

Также просто датафрэйм сохраняется на диск

In [ ]:
CSV.write("Apple_Golden_2.csv", apples)

Можно использовать словари, но по мнению переводчика, этот способ устарел

```julia
appledict = Dict(strip(name)=>col for (name, col) in zip(applecolnames, applecols))
```
>по крайней мере у меня этот код выкидывает ошибку

### План

Чтобы использовать нейрон с двумя входами, мы будем использовать только два признака (числа) для каждого изображения, скажем, столбцы 3 и 4: среднее количество красного и среднее количество зеленого; тогда каждая точка данных будет двухмерным вектором, а точки данных лежат в двухмерной плоскости. У нас будет много входных данных, помеченных индексом $ i $. Мы будем обозначать $ i$-ую точку данных как $\mathbf{x}^{(i)}$.

Цель состоит в том, чтобы наш нейрон взял одну точку на двумерной плоскости в качестве входных данных и вернул один выход, который **классифицирует** его как яблоко ($ 0 $) или банан ($ 1 $). Для этого он должен '**выучить**' правильные значения своих параметров $ \mathbf {w} $ и $ b $. 

Для этого обучения нам понадобятся **метки** для каждого признака, которые идентифицируют их как яблоко (0) или как банан (1). Эти метки, в свою очередь, позволят нам создать функцию потерь, которая научит наш алгоритм определять, является ли данный выбор параметров хорошей или плохой работой по классификации наших изображений фруктов. *Так что же нам осталось сделать?* Вышесказанное может показаться сложным, но, к счастью, мы можем разбить всё это на ряд действенных шагов:

1. Очистить наши входные данные (количество красного и зеленого), чтобы получить их в удобном формате;
2. Создать последовательность меток, которые мы можем использовать для определения правильных и неправильных классификаций;
3. Определить функцию потерь, которая содержит параметры;
4. Реализовать алгоритм, чтобы выбрать параметры для нашего нейрона, минимизируя функцию потерь по отношению к параметрам;
5. Использовать все вышеперечисленное, чтобы обучить наш нейрон классифицировать изображения!

#### Примечание: 

Обратите внимание, что *в целом мы не можем ожидать, что одного нейрона будет достаточно для классификации.* 

Если один нейрон изо всех сил пытается классифицировать наши изображения, нам, возможно, потребуется использовать более сложную структуру нейронной сети (что соответствует использованию более сложной функции).

## Очистка данных

Обычно бывает необходимо каким-то образом «очистить» данные, то есть предварительно обработать их, прежде чем их можно будет использовать для выполнения любой интересующей вас задачи.

Наше следующее *мета*-упражнение будет состоять в том, чтобы собрать все данные из столбцов 3 и 4 в *единственный* вектор Julia `x` (каждый элемент которого сам является вектором), а метки - в один вектор` y`. Давайте сделаем это в несколько шагов!

#### Упражнение 1

Во-первых, давайте потренируемся в захвате одного столбца в DataFrame. Чтобы захватить столбец, вы можете индексировать в «DataFrame» имя нужного столбца, передаваемого в виде символа. В Julia символами являются имена, перед которыми стоит `:`. Например, мы могли бы извлечь столбец «высоты» из «яблок» путем индексации в `apples` с символом `:height`:

```julia
apples[!,:height]
```

Получите столбец `red` из массива `apples`. Каков тип у возвращаемого объекта? Сколько у него записей?

A) Array, 5 <br>
B) DataArray, 5 <br>
C) Array, 64 <br>
D) DataArray, 64 <br>
E) Array, 492 <br>
F) DataArray, 492

#### Упражнение 2

Мы можем получить отдельную запись в DataFrame, указав индекс строки записи и символ столбца. Например, чтобы получить доступ к высоте 4-го изображения яблока, мы должны выполнить

```julia
apples[4, :height]
```

Сколько красного в 63-м изображении банана?

#### Упражнение 3

Мы хотим реорганизовать данные из 3-го и 4-го столбцов «яблок» и «бананов», чтобы поместить эти данные в один массив. Давайте начнем с организации данных из 3-го и 4-го столбцов «яблок» в один массив «x_apples». Создайте x_apples так, чтобы в x_apples был один элемент для каждого изображения в apple. 

$ i$-м элементом  в `x_apples` должен быть` Vector`, то есть 1D-массив, с двумя элементами - количеством красного и количеством синего в $ i$-ом изображении из `apples`. Аналогичным образом создайте `Array` ` x_bananas`, используя данные из `bananas`.

In [ ]:
x_apples = [ [apples[i, :red], apples[i, :green]] for i in eachindex(apples[!,:green]) ]

In [ ]:
x_bananas = [ [bananas[i, :red], bananas[i, :green]] for i in eachindex(bananas[!,:green]) ]

#### Упражнение 4

Далее мы хотим объединить элементы `x_apples` и` x_bananas` в один массив `xs`. `xs` должен содержать сначала все элементы` x_apples`, а затем все элементы `x_bananas`. Используйте функцию `vcat` для создания` xs`.

In [ ]:
xs=vcat(x_apples,x_bananas)

#### Упражнение 5

Если вы зашли так далеко, наши данные в том формате, который мы хотим изучить. Теперь нам нужны ярлыки! Мы хотим сохранить метку (либо `0`, либо` 1` для каждого изображения яблока или банана в нашем наборе данных в массиве `ys`. Создайте массив `ys`, где $ i$-й элемент равен `0`, если $ i$-й элемент` xs` является apple, и = ` 1`, если элемент $ i$-й в `xs` является бананом.

#### Упражнение 6

Добавьте точки данных для всех изображений яблок и бананов в наших наборах данных на график, используя `scatter`. Постройте точки данных для яблок одним цветом и используйте другой цвет для точек банановых данных. 

Подсказка: вы можете использовать функции `first` и` last`.

## «Учим» вручную

Интуитивно, глядя на график данных, мы видим, что должно быть «легко» найти функцию, которая разделяет данные на бананы с одной стороны и яблоки с другой: нам просто нужно нарисовать прямую линию, которая разделяет два облака данных. Мы можем сделать это «вручную» следующим образом.

В приведенном ниже коде нейрон изучит функцию вида $ \sigma (\mathbf {w} \cdot \mathbf {x} + b) $. Поскольку $ \sigma $ выглядит как гладкая версия пошаговой функции, мы можем думать о классификации $ \sigma $, основанной на том, меньше ли значение ее выходного аргумента, чем `0.5` или больше, чем` 0.5`.

**Игра**: используйте интерактивную визуализацию, чтобы найти подходящие значения $ \mathbf {w} $ и $ b $, чтобы гиперплоскость $ \sigma (w_1 x_1 + w_2 x_2 + b) = 0,5 $ разделяла данные. Это то же самое, что и гиперплоскость, для которой $ w_1 x_1 + w_2 x_2 + b = 0 $! (Обратите внимание, что таких значений много!)

Мы можем решить для $ x_2 $ используя

$$x_2 = -(w_1 x_1 + b) / w_2,$$

и использовать это, чтобы нарисовать соответствующую гиперплоскость.

In [ ]:
using Interact

In [ ]:
@manipulate for w1 in -2:0.01:3, w2 in -2:0.01:3, b in -2:0.01:3
    
    scatter(first.(x_apples), last.(x_apples), m=:cross, label="apples")
    scatter!(first.(x_bananas), last.(x_bananas), label="bananas")
    
    ylims!(0.3, 0.66)
    xlims!(0.45, 0.75)
    
    plot!(x -> -(w1*x + b) / w2)
end

## Как нейрон может *научиться* классифицировать данные?

Теперь мы готовы к нашему первому опыту **машинного обучения**: мы позволим нейрону учиться автоматически, обрабатывая данные и настраивая параметры модели соответствующим образом (процесс, который мы называем «обучение»)!

Для заданных значений параметров $ w_1 $, $ w_2 $ и $ b $ функция $ f_{\mathbf {w}, b} $ отображает вектор длины $ 2 $ в число от $ 0 $ до $ 1 $. 

Теперь мы хотим, чтобы нейрон *выучил* подходящие значения этих параметров. Мы хотим выяснить (узнать!) параметры, такие, чтоб $ f $ моделировала взаимосвязь между данными, которые мы исследовали. 

Таким образом, вход нейрона будет представлять собой вектор из двух частей информации об изображении; давайте назовем данные об $ i $-ом изображении $ \mathbf {x} ^ {(i)} $. Нам также дан ярлык с надписью, к какому типу фруктов он относится: $ 0 $ для яблока и $ 1 $ для банана; давайте назовем это *желаемым* выходным числом $ y ^ {(i)} $. 

Когда мы вводим $ i$-е данные $ \mathbf {x} ^ {(i)} $, мы хотим, чтобы нейрон выдал вывод, который *наиболее близок* к желаемому выводу $ y ^ {(i)} $; то есть он должен **минимизировать** среднеквадратичное расстояние 

$$ L_i = [f _ {\mathbf {w}, b} (\mathbf {x} ^ {(i)}) - y ^ {(i) }] ^ 2. $$

Однако теперь мы видим ключевое отличие от того, что мы делали ранее: нейрон должен изменять свои параметры таким образом, чтобы ему удавалось минимизировать это расстояние для *всех* входных данных одновременно! Как мы можем выразить это математически? Мы еще раз определяем функцию потерь, $ L (\mathbf {w}, b) $, которая говорит нам «насколько мы неправы, когда параметры принимают заданные значения, а затем **минимизируем** эту функцию потерь стараясь удовлетворить всем его параметрам.

Одним из способов учета всех данных за раз является использование функции потерь «среднеквадратичная ошибка», которая является средним (квадратом) по всем разностям между выходными данными сети, $ f _ {\mathbf { w}, b} (\mathbf {x} ^ {(i)}) $ на $ i$-ых данных и желаемом выходе $ y ^ {(i)} $:

$$L_\mathrm{total}(\mathbf{w}, b) = \frac{1}{N} \sum_i L_i = \frac{1}{N} \sum_i [f_{\mathbf{w}, b}(\mathbf{x}^{(i)}) - y^{(i)} ]^2,$$

где $ N $ - общее количество данных в обучающем наборе. Почему мы выбираем именно эту функцию потерь? Поскольку минимально возможное значение этой функции потерь составляет $ 0 $ (поскольку она представляет собой сумму квадратов), и это достигается только тогда, когда нейронная сеть точно предсказывает выходной сигнал. Если мы сможем найти способ минимизировать эту функцию потерь, мы подойдем как можно ближе к этому идеальному прогнозу. (В целом, однако, мы не сможем получить точный прогноз.)

## Минимизация функции потерь: *стохастический* градиентный спуск

Мы уже знаем, как минимизировать потери на компьютере: мы просто вычисляем градиент и делаем градиентный спуск! Но здесь мы сталкиваемся с проблемой: функция $ L_\mathrm {total} $ обычно имеет много terms (?слагаемых, минимумов), и поэтому вычисление градиента этой функции будет очень трудоемким. 

Вместо этого мы будем использовать метод, называемый *стохастическим* градиентным спуском. Здесь идея заключается в том, что мы не будем использовать функцию полной потери; вместо этого на каждом шаге мы выберем случайную точку данных с номером $ i $ и сделаем шаг градиентного спуска для функции частичных потерь $ L_i $ , соответствующей только этой точке данных.

**Упражнение 7:** 

Напишите функции для функции частичной потери `L (w, b, x, y)`. 

Для этого вспомним

$$
\mathbf{x} = \begin{pmatrix} x_1 \\ x_2 \end{pmatrix};
\qquad
\mathbf{w} = \begin{pmatrix} w_1 & w_2 \end{pmatrix};
\qquad
f_{\mathbf{w}, b}(\mathbf{x}) = \sigma(\mathbf{w} \cdot \mathbf{x} + b),$$

и объявите `f (x, w, b)` как в блокноте 8.

#### Упражнение 8

Напишите функцию для градиента `L`, т.е.` ∇L (w, b, x, y) `, относительно параметров $ (w_1, w_2, b) $, используя конечные разности. $ ∇L $ будет вектором с одним компонентом на параметр:

$$∇L = \left( \frac{\partial L}{\partial w_1}, \frac{\partial L}{\partial w_2}, \frac{\partial L}{\partial b} \right).$$

#### Упражнение 9

Реализуйте стохастический градиентный спуск в функции `stochastic_gradient_descent (L, w, b, xs, ys, N = 1000)`. Используйте её, чтобы минимизировать функцию $L_\mathrm{total}$.

Алгоритм: для каждого из `N` шагов случайным образом выберите индекс $ i $ в векторе` xs`, в котором хранятся данные вашего изображения. Рассчитайте градиент функции стоимости, $ L_i $, для этого изображения и обновите каждый из параметров, $ p_j $, из $ L_i $ в соответствии с

$$p_j = p_j - 0.01 * ∇L_j$$

(Здесь $ j $ обозначает параметр $ j ^ {th} $ в $ L $ и аналогично компоненту $ j ^ {th} $ в $ ∇L $.)

`stochastic_gradient_descent` должен вернуть обновленные значения для вектора $ \mathbf {w} $ и скаляра $b$.

Необязательно: Следите за значением $ L_\mathrm {total} $ с течением времени, если вы хотите визуализировать процесс обучения.

#### Упражнение 10

Используйте значения `w` и` b` из последнего упражнения, чтобы увидеть, как `f` классифицирует пару изображений в наборе данных. 

В частности, вычислите `f`, используя 1-е и 90-е изображения в` xs`. Для какого изображения вывод `f` ближе к значению его метки? 

A) Вывод `f` для 1-го изображения в` xs` ближе к его метке 

B) Вывод `f` для 90-го изображения в` xs` ближе к его метке.

#### Упражнение 11

Используйте функцию `maximum` для определения максимального квадрата расстояния прогноза от истинного значения. (Для каждого изображения эта формула имеет вид $ y_i - f_ {w, b} (x_i) $.)

#### Упражнение 12

Используйте `w` и` b` из стохастического градиентного спуска, чтобы нарисовать функцию, которую сеть изучила, как и раньше, в виде гиперплоскости $ w_1 x + w_2 y + b = 0 $. 

Наложите это на данные. Правильно ли разделяет эта гиперплоскость данные? (То есть, это данные для всех яблок на одной стороне линии, и данные для всех бананов на другой стороне линии?) 

A) Да 

B) Нет